In [3]:


train_dir = 'G:/signData'
#train_dir = '../data/signData'
output_dir = f"{train_dir}/nptxt"
json_folder_path = f'{train_dir}/train/label/landmark'
morpheme_path = f'{train_dir}/train/label/morpheme'

import os
import json
import numpy as np



In [ ]:
person_list=[{"0":"0"}]
person_blacklist=[]
start_person="11"
for person in os.listdir(json_folder_path):
    if int(person) < int(start_person):
        continue
    person_output_path = os.path.join(output_dir, str(int(person)))
    os.makedirs(person_output_path, exist_ok=True)
    count=1
    word_list=[]
    for word_coords,word_morpheme in zip(os.listdir(os.path.join(json_folder_path, person)),os.listdir(os.path.join(morpheme_path, person))):
        if "F" in word_morpheme:
            file_path = os.path.join(morpheme_path, person, word_morpheme)
            morpheme_file_path = os.path.join(morpheme_path, person, word_morpheme)
            with open(morpheme_file_path, 'r', encoding="UTF8") as morpheme_file:
                data = json.load(morpheme_file)
                try:
                    name = data['data'][0]['attributes'][0]['name']
                    # word_list.append({data['metaData']['name'][7:15]:name})
                except IndexError as e:
                    person_blacklist.append({person: data['metaData']['name'][7:15]})
                    name=False
                    # word_list.append({data['metaData']['name'][7:15]:None})
                    print(f"Error reading {morpheme_file_path}: {e}")
                    continue
        if "F" in word_coords and name is not False:
            wordCoordL = np.empty((0, 4, 5, 3))  
            wordCoordR = np.empty((0, 4, 5, 3))
            wordCoordP = np.empty((0, 3, 10))
            print(word_coords[11:15])
            for frame in os.listdir(os.path.join(json_folder_path, person, word_coords)):
                file_path = os.path.join(json_folder_path, person, word_coords, frame)
                try:
                    with open(file_path, 'r') as json_file:
                        data = json.load(json_file)
                        lh_points = data['people']['hand_left_keypoints_3d']
                        rh_points = data['people']['hand_right_keypoints_3d']
                        p_points = data['people']['pose_keypoints_3d']

                        preFrameCoordP = np.array([[(960 * p_points[i] + 960 - 420) / (1500 - 420), 
                                                    (1080 * p_points[i + 1] + 540) / 1080,
                                                    (p_points[32 + 2] - p_points[i + 2]) / 10]
                                                   for i in range(0, len(p_points), 4)], dtype=np.float32)

                        preFrameCoordL = np.array([[(960 * lh_points[i] + 960 - 420) / (1500 - 420),
                                                    (1080 * lh_points[i + 1] + 540) / 1080,
                                                    (lh_points[2] - lh_points[i + 2]) / 10]
                                                   for i in range(4, len(lh_points), 4)], dtype=np.float32)

                        preFrameCoordR = np.array([[(960 * rh_points[i] + 960 - 420) / (1500 - 420),
                                                    (1080 * rh_points[i + 1] + 540) / 1080,
                                                    (rh_points[2] - rh_points[i + 2]) / 10]
                                                   for i in range(4, len(rh_points), 4)], dtype=np.float32)

                        preFrameCoordP = preFrameCoordP[[i for i in range(19) if (0 <= i <= 7) or (17 <= i <= 18)]]

                        frameCoordL=preFrameCoordL.reshape(5,4,3).transpose(1,0,2)[::-1]
                        frameCoordR=preFrameCoordR.reshape(5,4,3).transpose(1,0,2)[::-1]
                        frameCoordP=preFrameCoordP.T
                        wordCoordL = np.append(wordCoordL, [frameCoordL], axis=0)
                        wordCoordR = np.append(wordCoordR, [frameCoordR], axis=0)
                        wordCoordP = np.append(wordCoordP, [frameCoordP], axis=0)
                except json.JSONDecodeError as e:
                    print(f"Error reading {file_path}: {e}")
                    break
            label=[word_num,int(person),name]
            word_output_path = os.path.join(person_output_path, f'{label[0]}.npz')
            np.savez(word_output_path, wordCoordL=wordCoordL, wordCoordR=wordCoordR, wordCoordP=wordCoordP, label=label)
            print(f"Saved {word_output_path}")
            count+=1
        else:
            continue


In [ ]:
def load_data(person,word):
    path=f"{output_dir}/{person}/{word}.npz"
    data = np.load(path)
    wordCoordL = data['wordCoordL']
    wordCoordR = data['wordCoordR']
    wordCoordP = data['wordCoordP']
    label = data['label']

    return wordCoordL, wordCoordR, wordCoordP, label


def load_word(person, start, num):
    words = []
    for WNum in range(start, start + num):
        wordCoordL, wordCoordR, wordCoordP, label = load_data(person, WNum)
        words.append([wordCoordL, wordCoordR, wordCoordP, label])
    return words


In [26]:
person_blacklist=[]
word_dict={}
num=1
for word_morpheme in os.listdir(os.path.join(morpheme_path, person)):
    if "F" in word_morpheme:
        file_path = os.path.join(morpheme_path, person, word_morpheme)
        morpheme_file_path = os.path.join(morpheme_path, person, word_morpheme)
        with open(morpheme_file_path, 'r', encoding="UTF8") as morpheme_file:
            data = json.load(morpheme_file)
            try:
                name = data['data'][0]['attributes'][0]['name']
                word_dict[name]=num
            except IndexError as e:
                person_blacklist.append({person: data['metaData']['name'][7:15]})
                name=False
                # word_list.append({data['metaData']['name'][7:15]:None})
                print(f"Error reading {morpheme_file_path}: {e}")
                continue
        num+=1   
with open('wordtonum.json', 'w') as json_file:
    json.dump(word_dict, json_file)

In [27]:
with open('wordtonum.json', 'r', encoding="UTF8") as json_file:
    data = json.load(json_file)
    print(data)
    

{'고민': 1, '뻔뻔': 2550, '수어': 3, '남아': 4, '눈': 1372, '독신': 6, '음료수': 7, '발가락': 8, '슬프다': 9, '자극': 10, '안타깝다': 1315, '어색하다': 12, '여아': 13, '외국인': 14, '영아': 15, '신사': 16, '뉴질랜드': 17, '나사렛대학교': 18, '알아서': 2038, '장애인': 20, '열아홉번째': 21, '침착': 22, '성실': 23, '학교연혁': 24, '싫어하다': 25, '급하다': 1111, '필기시험': 27, '병문안': 28, '검사': 1129, '결승전': 380, '낚시터': 31, '낚시대': 32, '당뇨병': 33, '독서': 34, '매표소': 35, '면역': 36, '감기': 37, '배드민턴': 38, '변비': 39, '병명': 40, '보건소': 41, '불면증': 42, '불행': 43, '붕대': 44, '사위': 45, '설사': 46, '성병': 47, '방충': 48, '소화제': 49, '손녀': 50, '손자': 51, '수면제': 52, '수집가': 53, '여행지': 54, '예식장': 55, '올림픽경기': 56, '회복': 965, '첫번째': 58, '운동경기': 59, '입원': 60, '재혼': 61, '진단서': 62, '축구장': 63, '치료': 64, '치료법': 65, '친아들': 66, '퇴원': 67, '한약': 68, '한약방': 69, '빈혈': 70, '화상': 71, '가래떡': 72, '고깃국': 73, '고추': 74, '고추가루': 75, '사골': 76, '배춧국': 77, '꽈배기': 78, '벌꿀': 79, '꿀물': 80, '냄비': 81, '찬물': 82, '다과': 83, '지방경찰청장': 84, '된장찌개': 85, '돼지고기': 86, '두부': 87, '딸기': 88, '떡국': 89, '라면': 90, '막걸리': 91, '무': 92, '밥그릇': 

In [25]:
with open('wordtonum.json', 'r', encoding="UTF8") as json_file:
    data = json.load(json_file)
    print(data['신부'])
    

2142
